<a href="https://colab.research.google.com/github/Harinilalitha/Login_Page/blob/master/4_2_project_NER_and_MRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pytesseract PyPDF2
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (11.2 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 131015 f

In [ ]:
import pytesseract
import PyPDF2
from PIL import Image
import torch
from transformers import BertForTokenClassification, BertTokenizer
import re

import pytesseract
import PyPDF2
from PIL import Image
from google.colab import files

# Function to extract text from PDF using PyPDF2
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

# Function to perform OCR on image using Tesseract
def ocr_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# Ask user to upload a file
print("Please upload a file:")
uploaded_file = files.upload()

combined_text=''

# Check if a file is uploaded
if uploaded_file:
    file_path = list(uploaded_file.keys())[0]
    # Check if the uploaded file is an image
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Perform OCR on image
        image_text = ocr_image(file_path)
        print("\nText extracted from image (OCR):")
        print(image_text)
        ocr_text=image_text
    # Check if the uploaded file is a PDF
    elif file_path.lower().endswith('.pdf'):
        # Extract text from PDF document
        pdf_text = extract_text_from_pdf(file_path)
        print("\nText extracted from PDF document:")
        print(pdf_text)
        ocr_text=pdf_text

    else:
        print("Unsupported file format. Please upload an image (PNG, JPG, JPEG) or a PDF.")

    # Ask user for a description (optional)
    description = input("\nEnter a description (optional): ")
    print("Description:", description if description else "No description provided")
    text=ocr_text+" " + description

else:
    print("No file uploaded. Please upload a file.")

# Extract entities
entities = extract_entities(combined_text)

# Print extracted entities
#print("\nEntities extracted:")
#for entity in entities:
#    print(entity)'''

Please upload a file:


No file uploaded. Please upload a file.


NameError: name 'extract_entities' is not defined

In [ ]:
'''
import argparse
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import requests
import asyncio
import aiohttp
import socket
import warnings; warnings.filterwarnings('ignore') # aiohttp produces deprecation warnings that don't concern us
#import nest_asyncio; nest_asyncio.apply() # necessary to run nested async loops in jupyter notebooks

# Use a variety of agents for our ClientSession to reduce traffic per agent
# This (attempts to) avoid a ban for high traffic from any single agent
# We should really use proxybroker or similar to ensure no ban
user_agents = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
        ]

def make_header():
    '''
    Chooses a random agent from user_agents with which to construct headers
    :return headers: dict: HTTP headers to use to get HTML from article URL
    '''
    # Make a header for the ClientSession to use with one of our agents chosen at random
    headers = {
            'User-Agent':random.choice(user_agents),
            }
    return headers

async def extract_by_article(url):
    '''
    Extracts all data from a single article
    :param url: string: URL to a single article (i.e. root pubmed URL + PMID)
    :return article_data: Dict: Contains all data from a single article
    '''
    conn = aiohttp.TCPConnector(family=socket.AF_INET)
    headers = make_header()
    # Reference our articles DataFrame containing accumulated data for ALL scraped articles
    global articles_data
    async with aiohttp.ClientSession(headers=headers, connector=conn) as session:
        async with semaphore, session.get(url) as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")
            # Get article abstract if exists - sometimes abstracts are not available (not an error)
            try:
                abstract_raw = soup.find('div', {'class': 'abstract-content selected'}).find_all('p')
                # Some articles are in a split background/objectives/method/results style, we need to join these paragraphs
                abstract = ' '.join([paragraph.text.strip() for paragraph in abstract_raw])
            except:
                abstract = 'NO_ABSTRACT'
            # Get author affiliations - sometimes affiliations are not available (not an error)
            affiliations = [] # list because it would be difficult to split since ',' exists within an affiliation
            try:
                all_affiliations = soup.find('ul', {'class':'item-list'}).find_all('li')
                for affiliation in all_affiliations:
                    affiliations.append(affiliation.get_text().strip())
            except:
                affiliations = 'NO_AFFILIATIONS'
            # Get article keywords - sometimes keywords are not available (not an error)
            try:
                # We need to check if the abstract section includes keywords or else we may get abstract text
                has_keywords = soup.find_all('strong',{'class':'sub-title'})[-1].text.strip()
                if has_keywords == 'Keywords:':
                    # Taking last element in following line because occasionally this section includes text from abstract
                    keywords = soup.find('div', {'class':'abstract' }).find_all('p')[-1].get_text()
                    keywords = keywords.replace('Keywords:','\n').strip() # Clean it up
                else:
                    keywords = 'NO_KEYWORDS'
            except:
                keywords = 'NO_KEYWORDS'
            try:
                title = soup.find('meta',{'name':'citation_title'})['content'].strip('[]')
            except:
                title = 'NO_TITLE'
            authors = ''    # string because it's easy to split a string on ','
            try:
                for author in soup.find('div',{'class':'authors-list'}).find_all('a',{'class':'full-name'}):
                    authors += author.text + ', '
                # alternative to get citation style authors (no first name e.g. I. Zenkov)
                # all_authors = soup.find('meta', {'name': 'citation_authors'})['content']
                # [authors.append(author) for author in all_authors.split(';')]
            except:
                authors = ('NO_AUTHOR')
            try:
                journal = soup.find('meta',{'name':'citation_journal_title'})['content']
            except:
                journal = 'NO_JOURNAL'
            try:
                date = soup.find('time', {'class': 'citation-year'}).text
            except:
                date = 'NO_DATE'

            # Format data as a dict to insert into a DataFrame
            article_data = {
                'url': url,
                'title': title,
                'authors': authors,
                'abstract': abstract,
                'affiliations': affiliations,
                'journal': journal,
                'keywords': keywords,
                'date': date
            }
            # Add dict containing one article's data to list of article dicts
            articles_data.append(article_data)

async def get_pmids(page, keyword):
    """
    Extracts PMIDs of all articles from a pubmed search result, page by page,
    builds a url to each article, and stores all article URLs in urls: List[string]
    :param page: int: value of current page of a search result for keyword
    :param keyword: string: current search keyword
    :return: None
    """
    # URL to one unique page of results for a keyword search
    page_url = f'{pubmed_url}+{keyword}+&page={page}'
    headers = make_header()
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(page_url) as response:
            data = await response.text()
            # Parse the current page of search results from the response
            soup = BeautifulSoup(data, "lxml")
            # Find section which holds the PMIDs for all articles on a single page of search results
            pmids = soup.find('meta',{'name':'log_displayeduids'})['content']
            # alternative to get pmids: page_content = soup.find_all('div', {'class': 'docsum-content'}) + for line in page_content: line.find('a').get('href')
            # Extract URLs by getting PMIDs for all pubmed articles on the results page (default 10 articles/page)
            for pmid in pmids.split(','):
                url = root_pubmed_url + '/' + pmid
                urls.append(url)

def get_num_pages(keyword):
    '''
    Gets total number of pages returned by search results for keyword
    :param keyword: string: search word used to search for results
    :return: num_pages: int: number of pages returned by search results for keyword
    '''
    # Return user specified number of pages if option was supplied
    if args.pages != None: return args.pages

    # Get search result page and wait a second for it to load
    # URL to the first page of results for a keyword search
    headers=make_header()
    search_url = f'{pubmed_url}+{keyword}'
    with requests.get(search_url,headers=headers) as response:
        data = response.text
        soup = BeautifulSoup(data, "lxml")
        num_pages = int((soup.find('span', {'class': 'total-pages'}).get_text()).replace(',',''))
        return num_pages # Can hardcode this value (e.g. 10 pages) to limit # of articles scraped per keyword

async def build_article_urls(keywords):
    """
    PubMed uniquely identifies articles using a PMID
    e.g. https://pubmed.ncbi.nlm.nih.gov/32023415/ #shameless self plug :)
    Any and all articles can be identified with a single PMID

    Async wrapper for get_article_urls, page by page of results, for a single search keyword
    Creates an asyncio task for each page of search result for each keyword
    :param keyword: string: search word used to search for results
    :return: None
    """
    tasks = []
    for keyword in keywords:
        num_pages = get_num_pages(keyword)
        for page in range(1,num_pages+1):
            task = asyncio.create_task(get_pmids(page, keyword))
            tasks.append(task)

    await asyncio.gather(*tasks)

async def get_article_data(urls):
    """
    Async wrapper for extract_by_article to scrape data from each article (url)
    :param urls: List[string]: list of all pubmed urls returned by the search keyword
    :return: None
    """
    tasks = []
    for url in urls:
        if url not in scraped_urls:
            task = asyncio.create_task(extract_by_article(url))
            tasks.append(task)
            scraped_urls.append(url)

    await asyncio.gather(*tasks)


if __name__ == "__main__":
    # Set options so user can choose number of pages and publication date range to scrape, and output file name
    parser = argparse.ArgumentParser(description='Asynchronous PubMed Scraper')
    parser.add_argument('--pages', type=int, default=None, help='Specify number of pages to scrape for EACH keyword. Each page of PubMed results contains 10 articles. \n Default = all pages returned for all keywords.')
    parser.add_argument('--start', type=int, default=2019, help='Specify start year for publication date range to scrape. Default = 2019')
    parser.add_argument('--stop', type=int, default=2020, help='Specify stop year for publication date range to scrape. Default = 2020')
    parser.add_argument('--output', type=str, default='articles.csv',help='Choose output file name. Default = "articles.csv".')
    args = parser.parse_args()
    if args.output[-4:] != '.csv': args.output += '.csv' # ensure we save a CSV if user forgot to include format in --output option
    start = time.time()
    # This pubmed link is hardcoded to search for articles from user specified date range, defaults to 2019-2020
    pubmed_url = f'https://pubmed.ncbi.nlm.nih.gov/?term={args.start}%3A{args.stop}%5Bdp%5D'
    # The root pubmed link is used to construct URLs to scrape after PMIDs are retrieved from user specified date range
    root_pubmed_url = 'https://pubmed.ncbi.nlm.nih.gov'
    # Construct our list of keywords from a user input file to search for and extract articles from
    search_keywords = []
    with open('keywords.txt') as file:
        keywords = file.readlines()
        [search_keywords.append(keyword.strip()) for keyword in keywords]
    print(f'\nFinding PubMed article URLs for {len(keywords)} keywords found in keywords.txt\n')
    # Empty list to store all article data as List[dict]; each dict represents data from one article
    # This approach is considerably faster than appending article data article-by-article to a DataFrame
    articles_data = []
    # Empty list to store all article URLs
    urls = []
    # Empty list to store URLs already scraped
    scraped_urls = []

    # We use asyncio's BoundedSemaphore method to limit the number of asynchronous requests
    #    we make to PubMed at a time to avoid a ban (and to be nice to PubMed servers)
    # Higher value for BoundedSemaphore yields faster scraping, and a higher chance of ban. 100-500 seems to be OK.
    semaphore = asyncio.BoundedSemaphore(100)

    # Get and run the loop to build a list of all URLs
    loop = asyncio.get_event_loop()
    loop.run_until_complete(build_article_urls(search_keywords))
    print(f'Scraping initiated for {len(urls)} article URLs found from {args.start} to {args.stop}\n')
    # Get and run the loop to get article data into a DataFrame from a list of all URLs
    loop = asyncio.get_event_loop()
    loop.run_until_complete(get_article_data(urls))

    # Create DataFrame to store data from all articles
    articles_df = pd.DataFrame(articles_data, columns=['title','abstract','affiliations','authors','journal','date','keywords','url'])
    print('Preview of scraped article data:\n')
    print(articles_df.head(5))
    # Save all extracted article data to CSV for further processing
    filename = args.output
    articles_df.to_csv(filename)
    print(f'It took {time.time() - start} seconds to find {len(urls)} articles; {len(scraped_urls)} unique articles were saved to {filename}')

'''


usage: colab_kernel_launcher.py [-h] [--pages PAGES] [--start START] [--stop STOP]
                                [--output OUTPUT]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-d0d8e165-a831-4df1-97c3-95987d413bd8.json


SystemExit: 2

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:00


In [ ]:
from Bio import Entrez #to access ncbi databases
import pandas as pd
import numpy as np

In [ ]:
#function is meant to perform a PubMed database search based on the provided query.
def search(query):
  Entrez.email = 'email@example.com' #email address to be used for accessing the NCBI
  handle = Entrez.esearch(db='pubmed',
  sort='relevance',
  retmax='250000', #Specifies the maximum number of results to return
  retmode='xml',
  term=query)
  results = Entrez.read(handle)
  return results

In [ ]:
studies = search('COVID-19')
studiesIdList = studies['IdList']

In [ ]:
def fetch_details(id_list):
  ids = ','.join(id_list)
  Entrez.email = 'email@example.com'
  handle = Entrez.efetch(db='pubmed',
                         retmode='xml',
                         id=ids)
  results = Entrez.read(handle)
  return results

In [ ]:
title_list = []
abstract_list = []
journal_list = []
language_list = []
pubdate_year_list = []
pubdate_month_list = []

studies = fetch_details(studiesIdList)
chunk_size = 10000

for chunk_i in range(0, len(studiesIdList), chunk_size):
    chunk = studiesIdList[chunk_i:chunk_i + chunk_size]
    papers = fetch_details(chunk)
    for i, paper in enumerate(papers['PubmedArticle']):
        title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])
        try:
            abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
        except:
            abstract_list.append('No Abstract')
        journal_list.append(paper['MedlineCitation']['Article']['Journal']['Title'])
        language_list.append(paper['MedlineCitation']['Article']['Language'][0])
        try:
            pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
        except:
            pubdate_year_list.append('No Data')
        try:
            pubdate_month_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Month'])
        except:
            pubdate_month_list.append('No Data')

df = pd.DataFrame(list(zip(
    title_list, abstract_list, journal_list, language_list, pubdate_year_list, pubdate_month_list
)),
                  columns=[
                      'Title', 'Abstract', 'Journal', 'Language', 'Year', 'Month'
                  ])


In [ ]:
df['Month'].replace('Jan', '01', inplace=True)
df['Month'].replace('Feb', '02', inplace=True)
df['Month'].replace('Mar', '03', inplace=True)
df['Month'].replace('Apr', '04', inplace=True)
df['Month'].replace('May', '05', inplace=True)
df['Month'].replace('Jun', '06', inplace=True)
df['Month'].replace('Jul', '07', inplace=True)
df['Month'].replace('Aug', '08', inplace=True)
df['Month'].replace('Sep', '09', inplace=True)
df['Month'].replace('Oct', '10', inplace=True)
df['Month'].replace('Nov', '11', inplace=True)
df['Month'].replace('Dec', '12', inplace=True)
df['Month'].replace('No Data', np.nan, inplace=True)

In [ ]:
df.head(5)

,Title,Abstract,Journal,Language,Year,Month
0,"Origin, transmission, diagnosis and management...",Coronavirus has emerged as a global health thr...,Postgraduate medical journal,eng,2020,12
1,Recent Developments on Therapeutic and Diagnos...,The ongoing pandemic of coronavirus disease 20...,The AAPS journal,eng,2021,01
2,COVID-19 diagnosis -A review of current methods.,A fast and accurate self-testing tool for COVI...,Biosensors & bioelectronics,eng,2021,01
3,Emerging COVID-19 variants and their impact on...,The emergence of novel and evolving variants o...,Annals of medicine,eng,2022,12
4,Resolution of coronavirus disease 2019 (COVID-...,Coronavirus disease 2019 (COVID-19) was first ...,Expert review of anti-infective therapy,eng,2020,12


In [ ]:
import pandas as pd
# Convert DataFrame to CSV
df.to_csv('Scrapdata.csv', index=False)

In [ ]:
df.head(4)

,Title,Abstract,Journal,Language,Year,Month
0,"Origin, transmission, diagnosis and management...",Coronavirus has emerged as a global health thr...,Postgraduate medical journal,eng,2020,12
1,Recent Developments on Therapeutic and Diagnos...,The ongoing pandemic of coronavirus disease 20...,The AAPS journal,eng,2021,01
2,COVID-19 diagnosis -A review of current methods.,A fast and accurate self-testing tool for COVI...,Biosensors & bioelectronics,eng,2021,01
3,Emerging COVID-19 variants and their impact on...,The emergence of novel and evolving variants o...,Annals of medicine,eng,2022,12


In [ ]:
import pandas as pd
data=pd.read_csv("/content/Scrapdata.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
data.head(5)

,Title,Abstract,Journal,Language,Year,Month
0,"Origin, transmission, diagnosis and management...",Coronavirus has emerged as a global health thr...,Postgraduate medical journal,eng,2020,12.0
1,Recent Developments on Therapeutic and Diagnos...,The ongoing pandemic of coronavirus disease 20...,The AAPS journal,eng,2021,1.0
2,COVID-19 diagnosis -A review of current methods.,A fast and accurate self-testing tool for COVI...,Biosensors & bioelectronics,eng,2021,1.0
3,Emerging COVID-19 variants and their impact on...,The emergence of novel and evolving variants o...,Annals of medicine,eng,2022,12.0
4,Resolution of coronavirus disease 2019 (COVID-...,Coronavirus disease 2019 (COVID-19) was first ...,Expert review of anti-infective therapy,eng,2020,12.0


In [ ]:
'''
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# Tokenization and Padding
max_len = 100  # Maximum sequence length
all_texts = df['Abstract'].tolist()

# Tokenize texts
tokenized_texts = [text.split() for text in all_texts]

# Flatten the tokenized texts
all_words = [word for sublist in tokenized_texts for word in sublist]

# Vocabulary and word counts
word_counts = Counter(all_words)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

# Word to index mapping
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}

# Index to word mapping
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Encode texts to sequences
sequences = [[word_to_idx[word] for word in text.split()] for text in all_texts]

# Pad sequences
padded_sequences = [seq[:max_len] + [0] * max(0, max_len - len(seq)) for seq in sequences]

# Convert to PyTorch tensors
X = torch.tensor(padded_sequences, dtype=torch.long)
y = torch.zeros(X.size(0))  # Dummy labels for this example

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the BiLSTM model
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out[:, -1, :])
        return self.sigmoid(out)

# Define model parameters
vocab_size = len(word_to_idx) + 1
embedding_dim = 100
hidden_dim = 64
output_dim = 1

# Instantiate the model
model = BiLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data Loader
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training the model
epochs = 3
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss}")
'''

'\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\nfrom sklearn.model_selection import train_test_split\nfrom torch.utils.data import Dataset, DataLoader\nfrom collections import Counter\n\n# Tokenization and Padding\nmax_len = 100  # Maximum sequence length\nall_texts = df[\'Abstract\'].tolist()\n\n# Tokenize texts\ntokenized_texts = [text.split() for text in all_texts]\n\n# Flatten the tokenized texts\nall_words = [word for sublist in tokenized_texts for word in sublist]\n\n# Vocabulary and word counts\nword_counts = Counter(all_words)\nvocab = sorted(word_counts, key=word_counts.get, reverse=True)\n\n# Word to index mapping\nword_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}\n\n# Index to word mapping\nidx_to_word = {idx: word for word, idx in word_to_idx.items()}\n\n# Encode texts to sequences\nsequences = [[word_to_idx[word] for word in text.split()] for text in all_texts]\n\n# Pad sequences\npadded_sequences = [seq[:max_len] + [0] * max(0, max_len -

In [ ]:
'''import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

df = pd.DataFrame(data)

# Prepare the data for training
# Tokenize and label each word in the abstract as "ENTITY" or "OTHER"
all_abstracts = " ".join(df['Abstract'])
tokenized_abstracts = all_abstracts.split()
word_counts = Counter(tokenized_abstracts)

# Define a simple labeling strategy (assign "ENTITY" label to rare words)
threshold = 1  # Adjust this threshold as needed
word_labels = {word: "ENTITY" if count <= threshold else "OTHER" for word, count in word_counts.items()}

# Create labeled sequences
labeled_sequences = [[(word, word_labels[word]) for word in abstract.split()] for abstract in df['Abstract']]

# Split data into training and testing sets
train_data, test_data = train_test_split(labeled_sequences, test_size=0.2, random_state=42)

# Define vocabulary and label mapping
word_to_idx = {word: idx + 1 for idx, (word, _) in enumerate(word_labels.items())}  # Reserve 0 for padding
label_to_idx = {"OTHER": 0, "ENTITY": 1}

# Convert sequences to indices
train_data_indices = [([(word_to_idx[word], label_to_idx[label]) for word, label in sequence]) for sequence in train_data]
test_data_indices = [([(word_to_idx[word], label_to_idx[label]) for word, label in sequence]) for sequence in test_data]

# Define dataset class
class EntityDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define the BiLSTM model
class BiLSTMEntityRecognition(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTMEntityRecognition, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out)
        return self.softmax(out)

# Model parameters
vocab_size = len(word_to_idx) + 1
embedding_dim = 100
hidden_dim = 64
output_dim = 2  # OTHER or ENTITY

# Instantiate the model
entity_model = BiLSTMEntityRecognition(vocab_size, embedding_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(entity_model.parameters(), lr=0.001)

# Prepare DataLoader
# Prepare DataLoader
# Prepare DataLoader
# Prepare DataLoader
def collate_fn(batch):
    inputs = [torch.tensor([seq_item[0] for seq_item in seq], dtype=torch.long) for seq in batch]
    labels = [torch.tensor([seq_item[1] for seq_item in seq], dtype=torch.long) for seq in batch]
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=0)
    return inputs, labels

train_loader = DataLoader(train_data_indices, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_data_indices, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Training the model
epochs = 10
entity_model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = entity_model(inputs)
        loss = criterion(outputs.view(-1, output_dim), labels.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss}")

# Evaluate the model on test data
entity_model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = entity_model(inputs)
        _, predicted = torch.max(outputs, dim=2)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.view(-1).size(0)

accuracy = correct_predictions / total_predictions
print(f"Accuracy on test data: {accuracy:.2f}")

def recognize_entities_from_input(user_input, model, word_to_idx, entity_labels):
    tokenized_input = user_input.split()
    indexed_input = [word_to_idx[word] if word in word_to_idx else 0 for word in tokenized_input]

    max_len = min(model.embedding.num_embeddings, len(indexed_input))
    padded_input = indexed_input[:max_len] + [0] * max(0, max_len - len(indexed_input))
    input_tensor = torch.tensor(padded_input, dtype=torch.long).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_labels = torch.argmax(outputs, dim=2).squeeze()

    entities = []
    current_entity = ""
    current_label = None
    for idx, label_idx in enumerate(predicted_labels):
        label = entity_labels[label_idx.item()]
        if label != "OTHER":
            if current_entity != "" and label != current_label:
                entities.append((current_entity.strip(), current_label))
                current_entity = ""
            current_entity += tokenized_input[idx] + " "
            current_label = label
        else:
            if current_entity != "":
                entities.append((current_entity.strip(), current_label))
                current_entity = ""
    if current_entity != "":
        entities.append((current_entity.strip(), current_label))

    return entities

# Example usage
user_input = "Coronavirus disease 2019 (COVID-19) was first detected in China in December, 2019, and declared as a pandemic by the World Health Organization (WHO) on March 11, 2020. The current management of COVID-19 is based generally on supportive therapy and treatment to prevent respiratory failure. The effective option of antiviral therapy and vaccination are currently under evaluation and development."
recognized_entities = recognize_entities_from_input(user_input, entity_model, word_to_idx, ["OTHER", "ENTITY"])
print("Recognized Entities:")
for entity, label in recognized_entities:
    print(f"Entity: {entity}, Label: {label}")

'''

Epoch 1/10, Loss: 173.2533546090126
Epoch 2/10, Loss: 173.1802560687065
Epoch 3/10, Loss: 173.10268932580948


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

# Sample data
data = {
    'Abstract': [
        "Coronavirus disease 2019 (COVID-19) was first detected in China in December, 2019, and declared as a pandemic by the World Health Organization (WHO) on March 11, 2020.",
        "The current management of COVID-19 is based generally on supportive therapy and treatment to prevent respiratory failure.",
        "The effective option of antiviral therapy and vaccination are currently under evaluation and development."
    ]
}

# Tokenization and Padding
max_len = 100  # Maximum sequence length
all_texts = data['Abstract']

# Tokenize texts
tokenized_texts = [text.split() for text in all_texts]

# Flatten the tokenized texts
all_words = [word for sublist in tokenized_texts for word in sublist]

# Vocabulary and word counts
word_counts = Counter(all_words)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)

# Word to index mapping
word_to_idx = {word: idx + 1 for idx, word in enumerate(vocab)}

# Index to word mapping
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Encode texts to sequences
sequences = [[word_to_idx[word] for word in text.split()] for text in all_texts]

# Pad sequences
padded_sequences = [seq[:max_len] + [0] * max(0, max_len - len(seq)) for seq in sequences]

# Convert to PyTorch tensors
X = torch.tensor(padded_sequences, dtype=torch.long)
y = torch.zeros(X.size(0))  # Dummy labels for this example

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define the BiLSTM model
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out[:, -1, :])
        return self.sigmoid(out)

# Define model parameters
vocab_size = len(word_to_idx) + 1
embedding_dim = 100
hidden_dim = 64
output_dim = 1

# Instantiate the model
model = BiLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data Loader
train_dataset = TextDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training the model
epochs = 3
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss}")

# Entity Recognition

# Prepare the data for training
# Tokenize and label each word in the abstract as "ENTITY" or "OTHER"
all_abstracts = " ".join(data['Abstract'])
tokenized_abstracts = all_abstracts.split()
word_counts = Counter(tokenized_abstracts)

# Define a simple labeling strategy (assign "ENTITY" label to rare words)
threshold = 1  # Adjust this threshold as needed
word_labels = {word: "ENTITY" if count <= threshold else "OTHER" for word, count in word_counts.items()}

# Create labeled sequences
labeled_sequences = [[(word, word_labels[word]) for word in abstract.split()] for abstract in data['Abstract']]

# Split data into training and testing sets
train_data, test_data = train_test_split(labeled_sequences, test_size=0.2, random_state=42)

# Define vocabulary and label mapping
word_to_idx_entity = {word: idx + 1 for idx, (word, _) in enumerate(word_labels.items())}  # Reserve 0 for padding
label_to_idx_entity = {"OTHER": 0, "ENTITY": 1}

# Convert sequences to indices
train_data_indices = [([(word_to_idx_entity[word], label_to_idx_entity[label]) for word, label in sequence]) for sequence in train_data]
test_data_indices = [([(word_to_idx_entity[word], label_to_idx_entity[label]) for word, label in sequence]) for sequence in test_data]

# Define dataset class for entity recognition
class EntityDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define the BiLSTM model for entity recognition
class BiLSTMEntityRecognition(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(BiLSTMEntityRecognition, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out)
        return self.softmax(out)

# Model parameters for entity recognition
vocab_size_entity = len(word_to_idx_entity) + 1
embedding_dim_entity = 100
hidden_dim_entity = 64
output_dim_entity = 2  # OTHER or ENTITY

# Instantiate the entity recognition model
entity_model = BiLSTMEntityRecognition(vocab_size_entity, embedding_dim_entity, hidden_dim_entity, output_dim_entity)

# Define loss function and optimizer for entity recognition
criterion_entity = nn.CrossEntropyLoss()
optimizer_entity = optim.Adam(entity_model.parameters(), lr=0.001)

# Prepare DataLoader
# Prepare DataLoader
def collate_fn(batch):
    inputs = [torch.tensor([seq_item[0] for seq_item in seq], dtype=torch.long) for seq in batch]
    labels = [torch.tensor([seq_item[1] for seq_item in seq], dtype=torch.long) for seq in batch]
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=0)
    return inputs, labels

train_loader_entity = DataLoader(train_data_indices, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Training the entity recognition model
epochs_entity = 10
entity_model.train()
for epoch in range(epochs_entity):
    running_loss = 0.0
    for inputs, labels in train_loader_entity:
        optimizer_entity.zero_grad()
        outputs = entity_model(inputs)
        loss = criterion_entity(outputs.view(-1, output_dim_entity), labels.view(-1))
        loss.backward()
        optimizer_entity.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs_entity}, Loss: {running_loss}")

# Evaluate the entity recognition model on test data
test_loader_entity = DataLoader(test_data_indices, batch_size=32, shuffle=False, collate_fn=collate_fn)

entity_model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for inputs, labels in test_loader_entity:
        outputs = entity_model(inputs)
        _, predicted = torch.max(outputs, 2)
        total_predictions += labels.view(-1).size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = correct_predictions / total_predictions
print(f"Accuracy on test data for entity recognition: {accuracy:.2f}")

# Define function to recognize entities from input text
def recognize_entities_from_input(user_input, model, word_to_idx, entity_labels):
    tokenized_input = user_input.split()
    indexed_input = [word_to_idx[word] if word in word_to_idx else 0 for word in tokenized_input]

    max_len = min(model.embedding.num_embeddings, len(indexed_input))
    padded_input = indexed_input[:max_len] + [0] * max(0, max_len - len(indexed_input))
    input_tensor = torch.tensor(padded_input, dtype=torch.long).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        predicted_labels = torch.argmax(outputs, dim=2).squeeze()

    entities = []
    current_entity = ""
    current_label = None
    for idx, label_idx in enumerate(predicted_labels):
        label = entity_labels[label_idx.item()]
        if label != "OTHER":
            if current_entity != "" and label != current_label:
                entities.append((current_entity.strip(), current_label))
                current_entity = ""
            current_entity += tokenized_input[idx] + " "
            current_label = label
        else:
            if current_entity != "":
                entities.append((current_entity.strip(), current_label))
                current_entity = ""
    if current_entity != "":
        entities.append((current_entity.strip(), current_label))

    return entities

# Example usage
user_input = "Coronavirus disease 2019 (COVID-19) was first detected in China in December, 2019, and declared as a pandemic by the World Health Organization (WHO) on March 11, 2020. The current management of COVID-19 is based generally on supportive therapy and treatment to prevent respiratory failure. The effective option of antiviral therapy and vaccination are currently under evaluation and development."
recognized_entities = recognize_entities_from_input(user_input, entity_model, word_to_idx_entity, ["OTHER", "ENTITY"])
print("Recognized Entities:")
for entity, label in recognized_entities:
    print(f"Entity: {entity}, Label: {label}")



Epoch 1/3, Loss: 0.6718016862869263
Epoch 2/3, Loss: 0.5921175479888916
Epoch 3/3, Loss: 0.5203841328620911
Epoch 1/10, Loss: 0.6932229399681091
Epoch 2/10, Loss: 0.6920270323753357
Epoch 3/10, Loss: 0.6904973387718201
Epoch 4/10, Loss: 0.6885781288146973
Epoch 5/10, Loss: 0.6873902082443237
Epoch 6/10, Loss: 0.6855071187019348
Epoch 7/10, Loss: 0.6839179396629333
Epoch 8/10, Loss: 0.682275116443634
Epoch 9/10, Loss: 0.6810504794120789
Epoch 10/10, Loss: 0.6788795590400696
Accuracy on test data for entity recognition: 0.67
Recognized Entities:
Entity: Coronavirus disease 2019 (COVID-19) was first, Label: ENTITY
Entity: in China in December,, Label: ENTITY
Entity: as a pandemic, Label: ENTITY
Entity: World, Label: ENTITY
Entity: Organization (WHO), Label: ENTITY
Entity: March 11, 2020., Label: ENTITY
Entity: current management, Label: ENTITY
Entity: is based generally, Label: ENTITY
Entity: supportive, Label: ENTITY
Entity: treatment to prevent respiratory failure., Label: ENTITY
Entity

In [ ]:
pip install transformers

In [ ]:
data=pd.read_csv("/content/Scrapdata.csv")

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example input sentences
sentences = ["This is the first sentence.",
             "Here is the second sentence.",
             "And this is the third sentence."]

# Tokenize input sentences and create attention masks
input_ids = []
attention_masks = []

for sentence in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sentence,                      # Sentence to encode
                        add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
                        max_length = 32,              # Pad & truncate all sentences
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True, # Construct attention masks
                        return_tensors = 'pt',        # Return PyTorch tensors
                   )

    # Add the encoded sentence to the list
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding)
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Verify the shapes of input tensors
print("Shape of input_ids:", input_ids.shape)
print("Shape of attention_masks:", attention_masks.shape)

# Define batch size and create DataLoaders
batch_size = 2  # Adjust batch size as needed
train_data = TensorDataset(input_ids, attention_masks)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Print example batches from the DataLoader
for step, batch in enumerate(train_dataloader):
    input_ids_batch, attention_masks_batch = batch
    print("Batch", step)
    print("Input IDs:", input_ids_batch)
    print("Attention Masks:", attention_masks_batch)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Shape of input_ids: torch.Size([3, 32])
Shape of attention_masks: torch.Size([3, 32])
Batch 0
Input IDs: tensor([[ 101, 2182, 2003, 1996, 2117, 6251, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 1998, 2023, 2003, 1996, 2353, 6251, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
Attention Masks: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
Batch 1
Input IDs: tensor([[ 101, 2023, 2003, 1996, 2034, 6251, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertTokenizer, BertModel


# Instantiate BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Example sentences
sentences = ["Coronavirus disease 2019 (COVID-19) was first detected in China.",
             "The ongoing pandemic of coronavirus disease 2019 (COVID-19) is caused by the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)."]

# Tokenize sentences and pad sequences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Get input IDs and attention masks
input_ids = inputs["input_ids"]
attention_masks = inputs["attention_mask"]

# Define batch size and create DataLoaders
batch_size = 32
train_data = TensorDataset(input_ids, attention_masks)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Model forward pass
for batch in train_dataloader:
    batch_input_ids, batch_attention_masks = batch
    outputs = bert_model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
    # Process model outputs as needed
    print(outputs)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0131, -0.2477, -0.1943,  ...,  0.0981,  0.0220,  0.5718],
         [-0.4217, -0.4706, -0.8185,  ...,  0.0962,  0.1811, -0.3232],
         [ 0.2471, -0.2295, -0.2051,  ..., -0.6135, -0.0994, -0.0831],
         ...,
         [ 0.6919, -0.0849, -0.2851,  ...,  0.0241, -0.7946, -0.2957],
         [ 0.5887, -0.1068, -0.3718,  ...,  0.1842, -0.6782, -0.3788],
         [-0.3277, -0.0380, -0.4514,  ...,  0.2994, -0.0083,  0.1644]],

        [[-0.4908, -0.5611, -0.7175,  ...,  0.0371,  0.0876,  0.3455],
         [ 0.8556, -0.2203, -0.1927,  ...,  0.3428,  0.5939,  0.3270],
         [ 0.6708, -0.1220, -0.6512,  ..., -0.0923,  0.1940, -0.1461],
         ...,
         [-0.0740, -0.6620,  0.1258,  ...,  0.0504, -0.1122,  0.4148],
         [-0.1952, -0.7806,  0.2138,  ...,  0.1999, -0.2303,  0.4887],
         [-0.4231, -0.5478,  0.1246,  ...,  0.1092, -0.4221,  0.3782]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_ou

In [ ]:
# Assume you have a list of labels corresponding to each token predicted by the model
predicted_labels = ["OTHER", "DRUG", "DRUG", "OTHER", "DISEASE", "OTHER", "DATE", "OTHER"]

# Convert tokenized text to original text
original_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)

# Define recognized entities
recognized_entities = []
current_entity = ""
current_label = None
medical_labels = ["DRUG", "DISEASE", "DATE"]  # Define medical-related labels

for idx, label in enumerate(predicted_labels):
    if label in medical_labels:
        if current_entity != "" and label != current_label:
            recognized_entities.append((current_entity.strip(), current_label))
            current_entity = ""
        current_entity += original_text.split()[idx] + " "
        current_label = label
    else:
        if current_entity != "":
            recognized_entities.append((current_entity.strip(), current_label))
            current_entity = ""

if current_entity != "":
    recognized_entities.append((current_entity.strip(), current_label))

# Print recognized entities
print("Recognized Entities:")
for entity, label in recognized_entities:
    print(f"Entity: {entity}, Label: {label}")


Recognized Entities:
Entity: disease 2019, Label: DRUG
Entity: covid, Label: DISEASE
Entity: 19, Label: DATE


In [1]:

git add .


SyntaxError: invalid syntax (<ipython-input-1-3d6cfb4a5e4d>, line 1)